<a href="https://colab.research.google.com/github/liadmagen/NLP-Course/blob/master/exercises_notebooks/02_LM_extracting_information_from_news_with_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

We start by importing pandas - a great tool for data scientists!

We load a csv of news articles from https://github.com/tblock/10kGNAD


In [ ]:
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/articles.csv', sep=';', error_bad_lines=False, header=None, names=['category', 'content'])

# First glance at the Data

let's take a look at our corpus. 

In [ ]:
df

,category,content
0,Etat,"Die ARD-Tochter Degeto hat sich verpflichtet, ..."
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,"'Zum Welttag der Suizidprävention ist es Zeit,..."
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service: Jobwechsel in der Kommunikationsbranc...
...,...,...
10268,Wissenschaft,Die Fundstelle in Südengland ist Unesco-Weltku...
10269,Wissenschaft,Im Team arbeitet auch ein Inspektor der sudane...
10270,Wissenschaft,Die zentrale Frage des Projekts: Siedelten Ägy...
10271,Wissenschaft,Klimatische Verschlechterungen dürften zur Auf...


We'll print the first row's content:

In [ ]:
df.iloc[0]['content']

'Die ARD-Tochter Degeto hat sich verpflichtet, ab August einer Quotenregelung zu folgen, die für die Gleichstellung von Regisseurinnen sorgen soll. In mindestens 20 Prozent der Filme, die die ARD-Tochter Degeto produziert oder mitfinanziert, sollen ab Mitte August Frauen Regie führen. Degeto-Chefin Christine Strobl folgt mit dieser Selbstverpflichtung der Forderung von Pro Quote Regie. Die Vereinigung von Regisseurinnen hatte im vergangenen Jahr eine Quotenregelung gefordert, um den weiblichen Filmschaffenden mehr Gehör und ökonomische Gleichstellung zu verschaffen. Pro Quote Regie kritisiert, dass, während rund 50 Prozent der Regie-Studierenden weiblich seien, der Anteil der Regisseurinnen bei Fernsehfilmen nur bei 13 bis 15 Prozent liege. In Österreich sieht die Situation ähnlich aus, auch hier wird von unterschiedlichen Seiten Handlungsbedarf angemahnt. Aber wie soll dieser aussehen? Ist die Einführung der Quotenregelung auch für die österreichische Film- und Fernsehlandschaft sinnv

We can see that the corpus, a german news collection. is divided into topics:

In [ ]:
df['category'].value_counts()

Panorama         1678
Web              1677
International    1511
Wirtschaft       1411
Sport            1201
Inland           1015
Etat              668
Wissenschaft      573
Kultur            539
Name: category, dtype: int64

Let's see the content of the Web news articles:

In [ ]:
df[df['category']=='Web']

,category,content
6612,Web,Android-Handy überzeugt mit solider Hardware u...
6613,Web,Spiel soll Mitte Juni ausführlich vorgestellt ...
6614,Web,"Von ""Leisure Suit Larry"" bis ""GTA"": Sex gibt e..."
6615,Web,Tyler Kirkham ist Comic-Zeichner und privat au...
6616,Web,Ohne einen Screenshot oder ein Gameplay-Video ...
...,...,...
8284,Web,"Liebeskummer, Zoten, Spritzwein – das wird in ..."
8285,Web,Hype um neue App des Vine-Mitbegründers. In de...
8286,Web,Wearables für Tiere: Smartbell statt tradition...
8287,Web,"Messaging vor Publikum, nicht nur mit Gespräch..."


We can see that the news are raw. Unlike with the NLTK, these are not divided into tokens, nor even divided into sentences.

# Regular Expressions

If you are not familiar with Regular Expressions (RegEx), please watch the following short explanation video:

https://www.youtube.com/watch?v=EyzTQ0OKeNw

Even nowadays, at the era of Neural Networks, RegEx are still being used often in NLP.
It's a great way to:
* Extract entities and sequential information out of the text: such as emails, dates, legal info such as law paragraph numbers, etc. 
* Extract candidates that can be later classified and pruned using a neural network

Please watch this short video, explaining how RegEx are used in NLP:
https://www.youtube.com/watch?v=sUNEGBuRWzU

Let's analyze our text using it.

In [ ]:
# let's convert first our articles to a list of strings:
web_articles = df[df['category']=='Web']['content'].to_list()
web_articles[:5]

['Android-Handy überzeugt mit solider Hardware und sehr guter Verarbeitung – Teilnahmeschluss 31.1.2016. Erst vor wenigen Wochen hat der WebStandard das OnePlus X getestet. Der Befund: Das Android-Smartphone liegt zwar etwas rutschig in der Hand, überzeugt aber sonst in fast allen Belangen. Insbesondere in puncto Verarbeitung vermag das Gerät zu glänzen. Teilnahme Nun gibt es ein OnePlus X (Glass-Edition) zu gewinnen. Um ein Los in den Topf zu werfen, haben Sie zwei Möglichkeiten, mit denen Sie maximal zwei Lose (eines pro Plattform) in den Gewinnspieltopf werfen können: 1) Sie liken oder kommentieren unseren Gewinnspielbeitrag auf Facebook oder 2) Sie folgen uns auf Twitter und teilen unseren Gewinnspielbeitrag dort. Bereitgestellt wird der Preis von OnePlus. Das OnePlus X wird Anfang Februar unter allen Teilnehmern verlost. Der/die Gewinner/in wird je nach Teilnahmeweg via Twitter oder Facebook verständigt. Teilnahmeschluss ist der 31. Jänner 2016. Keine Bargeldablöse. Der Rechtsweg 

With regex (package re in python) we can do searches after relevant entities, dates or numbers.

You can validate your regex on websites such as: https://regex101.com/

In [ ]:
import re

In [ ]:
re.findall(r'Android|OnePlus', web_articles[0])

['Android', 'OnePlus', 'Android', 'OnePlus', 'OnePlus', 'OnePlus']

In [ ]:
re.findall(r'\d+', web_articles[0])

['31', '1', '2016', '1', '2', '31', '2016']

In [ ]:
re.findall(r'\d+', web_articles[4])

['18', '4', '6', '2014', '4', '18', '500', '000', '2017', '4', '4']

In German numerical ammounts, the thousands are separated by a period '.', and a comma (',') separates the decimal part.

Instead of findal, we'll use 'search()' method, which also returns the location of the matched object in the string:

In [ ]:
### Your turn






### expected result: 
### found 500.000 at position 541 to 548)


found 500.000 at position 541 to 548)


In [ ]:
### Your turn

pattern = r''            # write a regex that extracts dates (of the format d.m.yy)





### expected result:
### found 31.1.2016 at position 92 to 101)

found 31.1.2016 at position 92 to 101)


Pandas has already regular expression built in. 
You can filter documents directly with it.

It also supports case insensitive search: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html

In [ ]:
df[df['content'].str.contains(r'Android|onePlus|iPhone')]

,category,content
10,Etat,News werden nach Interessen der Leser ausgelie...
18,Etat,Debatte über die Zukunft des Fernsehens bei Di...
167,Etat,Gründer der Satirezeitung schreibt ab sofort w...
274,Etat,"E-Papers von DER STANDARD, ""Die Presse"", ""Klei..."
771,Inland,Gerald Lembke über verbesserte Noten durch Lap...
...,...,...
8297,Wirtschaft,"Die ams-Aktien brechen am Mittwoch ein, das Un..."
8557,Wirtschaft,Hysterie oder berechtigte Sorge – wie ist die ...
8844,Wirtschaft,Volvo-Chef Hakan Samuelsson ist vor Apple und ...
9503,Wirtschaft,Nachdem Apple mehr als 300 Millionen Euro Steu...


using 'extract' method, we can extract groups (marked with parenthesis):

In [ ]:
df['content'].str.extract(r'(Android)|(onePlus)|(iPhone)').head(20)

,0,1,2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [ ]:
### Your turn: extract from the documents the date(s) that were found in the text and add it as a column, named 'date'
### Add another column, named 'amounts', that shows founded numerical amounts



Let's observe the extracted dates. Are all of them correctly extracted?

If not, how can you fix the regex to correctly capture them?

In [ ]:
df[~df['date'].isna()]

,category,content,date,amounts
33,Etat,Plus bei allen Messwerten im Vergleich zum Vor...,14.9.2015,"806.763,"
37,Etat,"Verlage müssen dort sein, wo die User sind. De...",19.6.2015,NaN
46,Etat,Andreas Koller verlangt von Medienminister Ost...,18.11.2015,NaN
83,Etat,Wie lecker ist Österreichisch?: Diskussion übe...,16.3.2016,NaN
99,Etat,Bei Ingrid Thurnher zu Gast waren Manfred Webe...,21.6.2015,NaN
...,...,...,...,...
10209,Wissenschaft,1830 – Die provisorische Regierung proklamiert...,4.10.2015,2.000
10211,Wissenschaft,1520 – Der portugiesische Seefahrer Magellan (...,23.10.1940,NaN
10215,Wissenschaft,1161 – Die Wiener Ruprechtskirche wird anlässl...,5.1.2016,NaN
10217,Wissenschaft,1516 – Nach dem Tod des Großvaters mütterliche...,23.1.2016,800.000


# Preprocessing

Simple tokenization, based on spaces, can also be done easily with regex.

In [ ]:
re.split(r'\W+', web_articles[0])[:10]

['Android',
 'Handy',
 'überzeugt',
 'mit',
 'solider',
 'Hardware',
 'und',
 'sehr',
 'guter',
 'Verarbeitung']

As you can see, it doesnt' take into considerations punctuations. Tokenizing a string with regex must then be improved, to be able to deal with these cases.

Try changing the regex expression to include also punctuations:

In [ ]:
### Your Turn: write a regex expression that extracts tokens with punctuations
### Hint: split might not be the best fit here
### Hint 2: nltk has a method - nltk.regexp_tokenize 








['Android-Handy',
 'überzeugt',
 'mit',
 'solider',
 'Hardware',
 'und',
 'sehr',
 'guter',
 'Verarbeitung',
 '–',
 'Teilnahmeschluss',
 '31',
 '.',
 '1',
 '.',
 '2016',
 '.',
 'Erst',
 'vor',
 'wenigen',
 'Wochen',
 'hat',
 'der',
 'WebStandard',
 'das',
 'OnePlus',
 'X',
 'getestet',
 '.',
 'Der',
 'Befund',
 ':',
 'Das',
 'Android-Smartphone',
 'liegt',
 'zwar',
 'etwas',
 'rutschig',
 'in',
 'der',
 'Hand',
 ',',
 'überzeugt',
 'aber',
 'sonst',
 'in',
 'fast',
 'allen',
 'Belangen',
 '.',
 'Insbesondere',
 'in',
 'puncto',
 'Verarbeitung',
 'vermag',
 'das',
 'Gerät',
 'zu',
 'glänzen',
 '.',
 'Teilnahme',
 'Nun',
 'gibt',
 'es',
 'ein',
 'OnePlus',
 'X',
 '(',
 'Glass-Edition',
 ')',
 'zu',
 'gewinnen',
 '.',
 'Um',
 'ein',
 'Los',
 'in',
 'den',
 'Topf',
 'zu',
 'werfen',
 ',',
 'haben',
 'Sie',
 'zwei',
 'Möglichkeiten',
 ',',
 'mit',
 'denen',
 'Sie',
 'maximal',
 'zwei',
 'Lose',
 '(',
 'eines',
 'pro',
 'Plattform',
 ')',
 'in',
 'den',
 'Gewinnspieltopf',
 'werfen',
 'könne

### A word about word-pieces

RegEx can be used to extract word-pieces.

Word-pieces are part of words that together construct the whole word. 

Remind yourself that we discussed **Morphology** - the study of structure of words, and which letters often 'go together. We mentioned that 'independently', for example can be breaken down to: 

**in** | **depend** | **ent** | **ly**.

In these cases, regex can help to match and find the patterns of the words.

### Tokenization

Let's use nltk to tokenize our raw text.

We start by importing the package:

In [ ]:
from nltk import word_tokenize

In [ ]:
web_articles[0]

'Android-Handy überzeugt mit solider Hardware und sehr guter Verarbeitung – Teilnahmeschluss 31.1.2016. Erst vor wenigen Wochen hat der WebStandard das OnePlus X getestet. Der Befund: Das Android-Smartphone liegt zwar etwas rutschig in der Hand, überzeugt aber sonst in fast allen Belangen. Insbesondere in puncto Verarbeitung vermag das Gerät zu glänzen. Teilnahme Nun gibt es ein OnePlus X (Glass-Edition) zu gewinnen. Um ein Los in den Topf zu werfen, haben Sie zwei Möglichkeiten, mit denen Sie maximal zwei Lose (eines pro Plattform) in den Gewinnspieltopf werfen können: 1) Sie liken oder kommentieren unseren Gewinnspielbeitrag auf Facebook oder 2) Sie folgen uns auf Twitter und teilen unseren Gewinnspielbeitrag dort. Bereitgestellt wird der Preis von OnePlus. Das OnePlus X wird Anfang Februar unter allen Teilnehmern verlost. Der/die Gewinner/in wird je nach Teilnahmeweg via Twitter oder Facebook verständigt. Teilnahmeschluss ist der 31. Jänner 2016. Keine Bargeldablöse. Der Rechtsweg i

...and tokenizing the text.

Notice that the tokens are more rich than the simple regex ones, and also, it tokenize all the sentences together - we don't have division by sentences. 

In [ ]:
tokens = word_tokenize(web_articles[0], language='german')
tokens[:50]

['Android-Handy',
 'überzeugt',
 'mit',
 'solider',
 'Hardware',
 'und',
 'sehr',
 'guter',
 'Verarbeitung',
 '–',
 'Teilnahmeschluss',
 '31.1.2016.',
 'Erst',
 'vor',
 'wenigen',
 'Wochen',
 'hat',
 'der',
 'WebStandard',
 'das',
 'OnePlus',
 'X',
 'getestet.',
 'Der',
 'Befund',
 ':',
 'Das',
 'Android-Smartphone',
 'liegt',
 'zwar',
 'etwas',
 'rutschig',
 'in',
 'der',
 'Hand',
 ',',
 'überzeugt',
 'aber',
 'sonst',
 'in',
 'fast',
 'allen',
 'Belangen.',
 'Insbesondere',
 'in',
 'puncto',
 'Verarbeitung',
 'vermag',
 'das',
 'Gerät']

We can load the tokens into nltk as an NLTK document.

In [ ]:
text = nltk.Text(tokens)
text[20:30]

['das',
 'OnePlus',
 'X',
 'getestet',
 '.',
 'Der',
 'Befund',
 ':',
 'Das',
 'Android-Smartphone']

With this nltk document, we can run analysis with all the nltk library commands:

In [ ]:
text.collocations()

unseren Gewinnspielbeitrag


In [ ]:
text.concordance('OnePlus')

Displaying 4 of 4 matches:
igen Wochen hat der WebStandard das OnePlus X getestet . Der Befund : Das Andro
glänzen . Teilnahme Nun gibt es ein OnePlus X ( Glass-Edition ) zu gewinnen . U
. Bereitgestellt wird der Preis von OnePlus . Das OnePlus X wird Anfang Februar
lt wird der Preis von OnePlus . Das OnePlus X wird Anfang Februar unter allen T


Let's use nltk to get the 20 most frequent tokens

In [ ]:
text.vocab().most_common(20)

[('.', 13),
 ('in', 5),
 ('der', 4),
 ('OnePlus', 4),
 (')', 4),
 ('Sie', 4),
 ('X', 3),
 (',', 3),
 ('zu', 3),
 ('oder', 3),
 ('wird', 3),
 ('überzeugt', 2),
 ('mit', 2),
 ('und', 2),
 ('Verarbeitung', 2),
 ('Teilnahmeschluss', 2),
 ('das', 2),
 ('Der', 2),
 (':', 2),
 ('Das', 2)]

### Stemming

Our text is not properly normalized, which makes it difficult to properly analyze. For example, in the most common list above, you can see repeated words. Can you spot which ones?

To clean up the text, we can lower all the tokens, remove stop-words and also use a process called '**Stemming**' and **Lemmatization**, to make similar words appear only once.

Read more about the different stemmers here:
https://www.nltk.org/api/nltk.stem.html

In [ ]:
import string

from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer

stemmer = GermanStemmer()

german_stop_words = set(stopwords.words('german'))
print(sorted(german_stop_words)[90:100])

['eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten']


In [ ]:
### Your turn: 
## - Join all the articles from our web_articles, separated with a new line (\n)
## - Lowercase the text
## - tokenize the text
## - stem all the tokens
## - remove all german stop words tokens 
## - remove all punctuation
## - Load it into NLTK library
## - Run the 20 most common tokens again








In [ ]:
text.vocab().most_common(20)

[('fur', 4660),
 ('werd', 2879),
 ('–', 2394),
 ('wurd', 1968),
 ('neu', 1874),
 ('uber', 1686),
 ('mehr', 1525),
 ('and', 1397),
 ('all', 1385),
 ('jahr', 1185),
 ('konn', 1170),
 ('nutz', 1107),
 ('hatt', 1079),
 ('appl', 985),
 ('erst', 983),
 ('allerding', 935),
 ('konnt', 926),
 ('spiel', 923),
 ('bereit', 910),
 ('weit', 844)]

One can also **Normalize** the text.

An example for a package that normalizes the text can be found here:
https://github.com/EFord36/normalise

However, it is only used for english texts.

Q: Can you create a similar package that works on your native language?

### Sentence Segmentation

We've **tokenied** the text into words. Now we also need to **segment** our sentences into groups. NLTK got us (almost) covered:

In [ ]:
nltk.sent_tokenize(web_articles[0], language='german')

['Android-Handy überzeugt mit solider Hardware und sehr guter Verarbeitung – Teilnahmeschluss 31.1.2016.',
 'Erst vor wenigen Wochen hat der WebStandard das OnePlus X getestet.',
 'Der Befund: Das Android-Smartphone liegt zwar etwas rutschig in der Hand, überzeugt aber sonst in fast allen Belangen.',
 'Insbesondere in puncto Verarbeitung vermag das Gerät zu glänzen.',
 'Teilnahme Nun gibt es ein OnePlus X (Glass-Edition) zu gewinnen.',
 'Um ein Los in den Topf zu werfen, haben Sie zwei Möglichkeiten, mit denen Sie maximal zwei Lose (eines pro Plattform) in den Gewinnspieltopf werfen können: 1) Sie liken oder kommentieren unseren Gewinnspielbeitrag auf Facebook oder 2) Sie folgen uns auf Twitter und teilen unseren Gewinnspielbeitrag dort.',
 'Bereitgestellt wird der Preis von OnePlus.',
 'Das OnePlus X wird Anfang Februar unter allen Teilnehmern verlost.',
 'Der/die Gewinner/in wird je nach Teilnahmeweg via Twitter oder Facebook verständigt.',
 'Teilnahmeschluss ist der 31.',
 'Jänner 2

Q: what actions could yout take to fix the wrong sentence divisions?

### Parts-of-speech

NLTK also offers **tagging** abilities. A **Tagger** classifies every word according to a given set of classes. For example - it can classifiy the tokens by their grammatical part-of-speech:

In [ ]:
nltk.pos_tag(word_tokenize(web_articles[0], language='german'), lang='deu', tagset='universal')

[('Android-Handy', 'ADJ'),
 ('überzeugt', 'NOUN'),
 ('mit', 'NOUN'),
 ('solider', 'NOUN'),
 ('Hardware', 'NOUN'),
 ('und', 'ADP'),
 ('sehr', 'ADJ'),
 ('guter', 'NOUN'),
 ('Verarbeitung', 'NOUN'),
 ('–', 'NOUN'),
 ('Teilnahmeschluss', 'NOUN'),
 ('31.1.2016', 'NUM'),
 ('.', '.'),
 ('Erst', 'NOUN'),
 ('vor', 'NOUN'),
 ('wenigen', 'NOUN'),
 ('Wochen', 'NOUN'),
 ('hat', 'PRON'),
 ('der', 'VERB'),
 ('WebStandard', 'NOUN'),
 ('das', 'NOUN'),
 ('OnePlus', 'NOUN'),
 ('X', 'NOUN'),
 ('getestet', 'NOUN'),
 ('.', '.'),
 ('Der', 'NOUN'),
 ('Befund', 'NOUN'),
 (':', '.'),
 ('Das', 'NOUN'),
 ('Android-Smartphone', 'NOUN'),
 ('liegt', 'NOUN'),
 ('zwar', 'NOUN'),
 ('etwas', 'NOUN'),
 ('rutschig', 'NOUN'),
 ('in', 'ADP'),
 ('der', 'NOUN'),
 ('Hand', 'NOUN'),
 (',', '.'),
 ('überzeugt', 'NOUN'),
 ('aber', 'NOUN'),
 ('sonst', 'NOUN'),
 ('in', 'ADP'),
 ('fast', 'ADJ'),
 ('allen', 'ADJ'),
 ('Belangen', 'NOUN'),
 ('.', '.'),
 ('Insbesondere', 'NOUN'),
 ('in', 'ADP'),
 ('puncto', 'NOUN'),
 ('Verarbeitung', 'N

Your turn: 

Write a sentence in your own nativa language, and try to tokenize and tag it using NLTK:

In [ ]:
### Your Turn:
### Write a sentence in your own nativa language, and try to tokenize and tag it using NLTK.
### Follow the package download instructions if necessary



Additional reading:

https://www.sussex.ac.uk/webteam/gateway/file.php?name=essay---parts-of-speech.pdf&site=1